Loading the model in 

In [ ]:
import torch
from common_utils import MLPBuilder

In [ ]:
# we trained the model with 8 features
model = MLPBuilder(no_features=8, layers=[256,128,32])

model.load_state_dict(torch.load('models/NN_model.pth'))